In [1]:
import numpy as np


In [2]:
def process_data(dataname):
    #reads in files, produces data structure with all actions
        #does so by applying produce_rule_list to every sentence.
        #for loop that sets actions to empty, calls p_r_l giving it
        #the stack and buffer, actions and correct_parse, adds finished action list
        #to new data file, for each sentence in the input data
    #input: name of the data file with all parses. Run with data file in same directory.
    #output: data file with all actions
    file = open(dataname)
    data = file.read()
    correct_parses = correct_parse_list(data)
    #gets rid of final whitespace
    del correct_parses[len(correct_parses)-1]
    
    #iterates over all parses, producing action list for each
    complete_rule_list = []
    for sentence_parse in correct_parses:
        stack = []
        print(len(sentence_parse))
        buff = list(range(1,len(sentence_parse)+1))
        actions = []
        rule_list = produce_rule_list(stack, buff, actions, sentence_parse)
        complete_rule_list.append(rule_list)

    
    return complete_rule_list

def correct_parse_list(data):
    #Turns data into a list of lists of lists with relevant information
    correct_parse = data.split("\n\n")
    for index, paragraph in enumerate(correct_parse):
        correct_parse[index] = paragraph.split("\n")
    for paragraph in correct_parse:
        for index, line in enumerate(paragraph):
            paragraph[index] = line.split("\t")
    return correct_parse



In [8]:
def produce_rule_list(stack, buff, actions, sentence_parse):
    #recursive function that works through words in the sentence (stack/buffer)
        #until only one word is left, creating the list of actions 
        #that was taken to parse it.
    #input: stack, buffer, actions, correct parse
    #output: actions with the actions taken for each buff/stack configuration
    
    #base case
    if len(stack) == 1 and len(buff) == 0:
        #actions.append([stack[:], "empty", "R_arc"])
        actions.append([stack[0],-1, -1, "R_root"])
        return actions

    #If enough of the sentence is still left:
    #If there is not enough material in the stack, shift:
    if len(stack) == 0 :
        #print('chose S - small stack')
        actions.append([-1,-1,buff[0], "S"])
        stack.append(buff[0])
        del buff[0]        
        return produce_rule_list(stack,buff,actions,sentence_parse)
    if len(stack) == 1:
        actions.append([-1,stack[-1],buff[0], "S"])
        stack.append(buff[0])
        del buff[0]
        return produce_rule_list(stack,buff,actions,sentence_parse)
    #If there are 2 or more words in the stack, decide which action to perform and perform it
    if len(stack) > 1:
        action = rule_decision(stack,buff,sentence_parse)
        stack, buff, actions = action(stack,buff,actions, sentence_parse)
        return produce_rule_list(stack,buff,actions,sentence_parse)
    

def rule_decision(stack, buff, sentence_parse):
    #determines which action to apply
    #input: words on stack, words on buff, correct parse
    #output: one of three methods, Shift(), L_arc(), R_arc()

    #TODO: find ids/heads (index [6]) from stack and sentence_parse
    s1 = stack[-2]
    head_of_s1 = int(sentence_parse[s1-1][6])
    s2 = stack[-1]
    head_of_s2 = int(sentence_parse[s2-1][6])
    
    #L arcs can always be applied if possible
    if head_of_s1 == s2:
        action = L_arc
        #print('chose L')
    else:
        #R arcs can only be applied if there is no word in the buffer which has the last word in the stack as a head
        if head_of_s2 == s1:
            buff_heads = [int(sentence_parse[x-1][6]) for x in buff]
            if s2 in buff_heads:
                action = Shift
                #print('chose S - s2 in buffheads')
            else:
                action = R_arc
                #print('chose R')
        #if there is no match between s1 and s2, simply shift another word from the buffer
        else:
            action = Shift
            #print('chose S - no matching s1s2')

    return action

#The following methods perform an arc or shift. These can be changed if more data is needed in the network.

def L_arc(stack, buff, actions, sentence_parse):
    #removes second to last item from stack, writes action to actions
    #input: stack and actions
    #output: new stack and actions with one L_arc line
    #s1, s2, b1, action
    s1 = int(stack[-2])
    s2 = int(stack[-1])
    b1 = int(stack[0])
    relation = sentence_parse[s1-1][7]
    #actions.append([stack[:], buff[:], "L_arc"])
    actions.append([s1,s2,b1, "L"+"_"+relation])
    del stack[-2]
    return stack, buff, actions



def R_arc(stack, buff, actions, sentence_parse):
    #removes last item from the stack, writes action to actions
    #input: stack and actions
    #output: new stack and actions with one R_arc line
    #s1, s2, b1, action
    s1 = int(stack[-2])
    s2 = int(stack[-1])
    b1 = int(stack[0])
    relation = sentence_parse[s2-1][7]
    #actions.append([stack[:], buff[:], "R_arc"])
    actions.append([s1,s2,b1, "R"+"_"+relation])
    del stack[-1]
    return stack, buff, actions



def Shift(stack, buff, actions, sentence_parse):
    #moves an item from the buff to the stack, writes action to actions
    #input: stack, buff and actions
    #output: new stack and actions with one extra shift line
    #s1, s2, b1, action
    s1 = int(stack[-2])
    s2 = int(stack[-1])
    b1 = int(stack[0])
    #actions.append([stack[:], buff[:], "Shift"])
    actions.append([s1,s2,b1, "S"])
    stack.append(buff[0])
    del buff[0]
    return stack, buff, actions

IndexError: list index out of range

In [9]:

action_data = process_data('train-stanford-raw.conll')

49
5
20
12
29
15
9
8
17
25
33
37
17
48
44
13
30
18
46
35
15
9
48
50
18
10
24
19
22
35
40
22
21
18
23
29
33
19
18
10
36
27
22
21
26
23
34
23
33
21
29
24
44
11
43
58
23
18
12
21
23
58
33
41
31
23
50
7
46
52
34
50
25
43
12
15
9
53
18
32
49
33
33
8
39
4
6
36
20
11
29
19
20
19
17
18
27
47
29
28
48
19
32
29
28
15
34
11
38
25
22
35
32
16
15
26
39
11
27
16
27
16
11
22
28
45
23
26
21
16
30
19
20
35
11
36
23
47
25
15
28
15
23
32
27
26
34
24
38
33
21
31
31
33
24
28
17
41
29
24
21
28
18
27
18
19
59
38
13
19
56
17
37
15
37
19
25
31
27
18
14
38
40
9
16
29
32
13
31
25
46
17
20
21
14
24
20
23
36
8
2
26
22
43
29
17
22
21
18
7
47
21
18
20
19
43
25
34
18
16
28
16
21
58
6
19
27
15
14
14
5
36
33
34
15
12
26
16
19
40
20
38
15
27
5
4
14
23
5
31
17
10
19
18
27
4
7
13
29
25
15
10
30
5
17
19
24
26
7
6
14
5
11
16
25
5
21
36
19
28
12
32
20
15
30
33
6
23
7
14
24
17
10
6
15
10
11
59
33
29
28
7
22
33
12
64
32
21
24
15
47
23
20
6
45
6
9
20
30
12
15
13
8
22
15
26
15
27
27
33
22
13
18
32
32
18
33
26
19
31
25
28
32
14
1

In [10]:
for action in action_data[-1]:
    print(action)

[-1, -1, 1, 'S']
[-1, 1, 2, 'S']
[1, 2, 1, 'S']
[2, 3, 1, 'L_aux']
[1, 3, 1, 'L_nsubj']
[-1, 3, 4, 'S']
[3, 4, 3, 'R_iobj']
[-1, 3, 5, 'S']
[3, 5, 3, 'S']
[5, 6, 3, 'L_det']
[3, 6, 3, 'S']
[6, 7, 3, 'S']
[7, 8, 3, 'L_aux']
[6, 8, 3, 'S']
[8, 9, 3, 'S']
[9, 10, 3, 'L_det']
[8, 10, 3, 'R_dobj']
[6, 8, 3, 'S']
[8, 11, 3, 'R_cc']
[6, 8, 3, 'S']
[8, 12, 3, 'S']
[12, 13, 3, 'L_advmod']
[8, 13, 3, 'S']
[13, 14, 3, 'S']
[14, 15, 3, 'S']
[15, 16, 3, 'L_amod']
[14, 16, 3, 'L_det']
[13, 16, 3, 'R_dobj']
[8, 13, 3, 'R_conj']
[6, 8, 3, 'R_infmod']
[3, 6, 3, 'R_dobj']
[-1, 3, 17, 'S']
[3, 17, 3, 'R_punct']
[3, -1, -1, 'R_root']


In [16]:
print(len(action_data))

39832
